# wps_convolution

#### wps_convolution is a process that runs the [convolution](https://rvic.readthedocs.io/en/latest/user-guide/convolution/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [ ]:
from birdy import WPSClient
import os
from datetime import datetime
from wps_tools.testing import get_target_url
from netCDF4 import Dataset
from rvic.core.config import read_config
from importlib.resources import files

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("osprey")
print(f"Using osprey on {url}")

Using osprey on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps


In [3]:
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
osprey.convolution?

Signature:
osprey.convolution(
    case_id,
    run_startdate,
    stop_date,
    domain=None,
    param_file=None,
    input_forcings=None,
    loglevel='INFO',
    convolve_config_file=None,
    convolve_config_dict=None,
    output_formats=None,
)
Docstring:
Aggregates the flow contribution from all upstream grid cellsat every timestep lagged according the Impuls Response Functions.

Parameters
----------
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
case_id : string
    Case ID for the RVIC process
run_startdate : string
    Run start date (yyyy-mm-dd-hh). Only used for startup and drystart runs.
stop_date : string
    Run stop date based on STOP_OPTION
domain : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Path to CESM complaint domain file
param_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Path to RVIC parameter file
input_forcings : ComplexData:mim

#### We can use the docstring to ensure we provide the appropriate parameters.

In [ ]:
# run convolution only with required inputs
output_basic = osprey.convolution(
    case_id = "sample",
    run_startdate = "2012-12-01-00",
    stop_date = "2012-12-31",
    domain = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_routing_domain.nc",
    param_file = str((files("tests") / "data/samples/sample.rvic.prm.COLUMBIA.20180516.nc").resolve()),
    input_forcings = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/columbia_vicset2.nc",
)
# Use asobj=True to access the data from the output file
basic_data = output_basic.get(asobj=True)[0]

In [ ]:
# run convolution with local configuration file
output_config_file = osprey.convolution(
    case_id = "sample",
    run_startdate = "2012-12-01-00",
    stop_date = "2012-12-31",
    domain = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_routing_domain.nc",
     param_file = str((files("tests") / "data/samples/sample.rvic.prm.COLUMBIA.20180516.nc").resolve()),
    input_forcings = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/columbia_vicset2.nc",
    convolve_config_file = str((files("tests") / "data/configs/convolve.cfg").resolve()),
)
config_file_data = output_config_file.get(asobj=True)[0]

In [ ]:
# run convolution with configuration dictionary
output_config_dict = osprey.convolution(
    case_id = "sample",
    run_startdate = "2012-12-01-00",
    stop_date = "2012-12-31",
    domain = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_routing_domain.nc",
    param_file = str((files("tests") / "data/samples/sample.rvic.prm.COLUMBIA.20180516.nc").resolve()),
    input_forcings = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/columbia_vicset2.nc",   
    convolve_config_dict = {
        "OPTIONS": {
            "CASESTR": "Historical",
        },
    }
)
config_dict_data = output_config_dict.get(asobj=True)[0]

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [8]:
# Expected output metadata determined from the input configuration file
expected = {
    "title": "RVIC history file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d").zfill(2),
    "RvicPourPointsFile": "sample_pour.txt",
    "RvicUHFile": "uhbox.csv",
    "RvicFdrFile": "sample_flow_parameters.nc",
    "RvicDomainFile": "sample_routing_domain.nc",
}

In [9]:
def test_convolution_output(output_data, expected):
    # Metadata of the output of RVIC convolution module
    metadata = {
        "title": output_data.title,
        "year": output_data.history.split()[5],
        "month": output_data.history.split()[2].zfill(2),
        "day": output_data.history.split()[3].zfill(2),
        "RvicPourPointsFile": output_data.RvicPourPointsFile,
        "RvicUHFile": output_data.RvicUHFile,
        "RvicFdrFile": output_data.RvicFdrFile,
        "RvicDomainFile": output_data.RvicDomainFile,
    }
    assert metadata == expected

In [10]:
test_convolution_output(basic_data, expected)
test_convolution_output(config_file_data, expected)
test_convolution_output(config_dict_data, expected)